# Data Collection
- Get list of 20 most visited cities.
- Create a dictionary with coordinates of 20 most visited cities.

In [1]:
import tqdm
import urllib
import pandas as pd
import requests
from scrapy import Selector
import json
import custom_functions as cf
import openmeteo_requests
import requests_cache
from retry_requests import retry
from sqlalchemy import create_engine

c:\Users\dunan\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\dunan\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
from geopy.geocoders import Nominatim


### Get a list of the 20 most visited cities:

In [2]:
cities_url = "https://travelness.com/most-visited-cities-in-the-world" # URL of the page with the list of cities

response = requests.get(cities_url)
sel = Selector(response)

cities = sel.xpath("//table//tr/td[2]/text()").getall()

### Get the City Stereotypes about weather for these 20 cities

In [3]:
suggestion_list = cf.extract_words(cities)

In [4]:
filtered_weather_words = cf.filter_weather_words(suggestion_list)

In [5]:
# Export the dict to a json file
with open('./../data/filtered_weather_words.json','w') as file:
    json.dump(filtered_weather_words,file)

### Use OpenStreetMaps API to convert the city names into coordinates

In [3]:
def geocode_city(city):
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(city)
    return {"city": city, "latitude": location.latitude, "longitude": location.longitude}

def geocode_cities(city_list):
    geocoded_cities = [geocode_city(city) for city in city_list if geocode_city(city)]
    return geocoded_cities

# Geocode the list of cities
geocoded_cities = geocode_cities(cities)

In [4]:
# export geocoded cities to a json file
with open("../data/city_coordinates.json", "w") as f:
    json.dump(geocoded_cities, f)

### Use the open-meteo API to get weather data

In [5]:
# Define our variables of interest as a list
daily_variables_of_interest = [
    "temperature_2m_max",
    "temperature_2m_min",
    "temperature_2m_mean",
    "daylight_duration",
    "sunshine_duration",
    "precipitation_sum",
    "rain_sum",
    "precipitation_hours",
]

# Create the API params dictionary
params = {
    "latitude": [city["latitude"] for city in geocoded_cities],
    "longitude": [city["longitude"] for city in geocoded_cities],
    "start_date": "1940-01-01",
    "end_date": "2023-12-31",
    "daily": daily_variables_of_interest,
}

In [6]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
responses = openmeteo.weather_api(url, params=params)

In [7]:
# List comprehension to create a list of dataframes
dataframes_list = [cf.process_response(response, geocoded_cities, i) for i, response in enumerate(responses)]

In [8]:
merged_df = pd.concat(dataframes_list, ignore_index=True)
merged_df.to_csv("../data/weather_data.csv", index=False)

### Scrape the NGRAM data into dataframes

In [13]:
#Collecting percentages for rain data
NGRAMSrain_df = pd.DataFrame()
queries_rain = ["London rain", "London Rain", "rainy London", "rain in London", "Rain in London", "raining in London", "Raining in London"]

for query in queries_rain:
    df_queries_rain = cf.runQuery(query)
    NGRAMSrain_df = pd.concat([NGRAMSrain_df, df_queries_rain])

NGRAMSrain_df.reset_index(drop=True, inplace=True)
NGRAMSrain_df.rename(columns={'Absolute Appearance %': 'Rain Absolute Appearance %'}, inplace=True)

#Summing the appearances into one dataframe
NGRAMSrain_df_grouped = NGRAMSrain_df.groupby('Year').sum().reset_index()
NGRAMSrain_df_grouped = NGRAMSrain_df_grouped.loc[:, ['Year', 'Rain Absolute Appearance %']]

# Create a new column by dividing each item in the 'Appearances' column by the first item in the column
NGRAMSrain_df_grouped['Rain Relative Appearance %'] = NGRAMSrain_df_grouped['Rain Absolute Appearance %'].apply(lambda x: x / NGRAMSrain_df_grouped['Rain Absolute Appearance %'].iloc[0])


In [14]:
#Collecting percentages for sun data
NGRAMSsun_df = pd.DataFrame()
queries_sun = ["London sun", "London Sun", "sunny London", "sun in London", "Sun in London"]

for query in queries_sun:
    df_queries_sun = cf.runQuery(query)
    NGRAMSsun_df = pd.concat([NGRAMSsun_df, df_queries_sun])

NGRAMSsun_df.reset_index(drop=True, inplace=True)
NGRAMSsun_df.rename(columns={'Absolute Appearance %': 'Sun Absolute Appearance %'}, inplace=True)

#Summing the appearances into one dataframe
NGRAMSsun_df_grouped = NGRAMSsun_df.groupby('Year').sum().reset_index()
NGRAMSsun_df_grouped = NGRAMSsun_df_grouped.loc[:, ['Year', 'Sun Absolute Appearance %']]

# Create a new column by dividing each item in the 'Appearances' column by the first item in the column
NGRAMSsun_df_grouped['Sun Relative Appearance %'] = NGRAMSsun_df_grouped['Sun Absolute Appearance %'].apply(lambda x: x / NGRAMSsun_df_grouped['Sun Absolute Appearance %'].iloc[0])


In [15]:
#Collecting percentages for wind data
NGRAMSwind_df = pd.DataFrame()
queries_wind = ["London wind", "London Wind", "windy London", "wind in London", "Wind in London"]

for query in queries_wind:
    df_queries_wind = cf.runQuery(query)
    NGRAMSwind_df = pd.concat([NGRAMSwind_df, df_queries_wind])

NGRAMSwind_df.reset_index(drop=True, inplace=True)
NGRAMSwind_df.rename(columns={'Absolute Appearance %': 'Wind Absolute Appearance %'}, inplace=True)

#Sum the appearances into one dataframe
NGRAMSwind_df_grouped = NGRAMSwind_df.groupby('Year').sum().reset_index()
NGRAMSwind_df_grouped = NGRAMSwind_df_grouped.loc[:, ['Year', 'Wind Absolute Appearance %']]

# Create a new column by dividing each item in the 'Appearances' column by the first item in the column
NGRAMSwind_df_grouped['Wind Relative Appearance %'] = NGRAMSwind_df_grouped['Wind Absolute Appearance %'].apply(lambda x: x / NGRAMSwind_df_grouped['Wind Absolute Appearance %'].iloc[0])


In [17]:
# Merge all the NGRAMS dataframes together
NGRAMS_df_grouped = pd.merge(NGRAMSrain_df_grouped, NGRAMSsun_df_grouped, on='Year')
NGRAMS_df_grouped = pd.merge(NGRAMS_df_grouped, NGRAMSwind_df_grouped, on='Year')

# Convert NGRAM_df_grouped into a csv file
NGRAMS_df_grouped.to_csv('../data/perception_data.csv', index=False)

# Display NGRAM_df_grouped
NGRAMS_df_grouped

,Year,Rain Absolute Appearance %,Rain Relative Appearance %,Sun Absolute Appearance %,Sun Relative Appearance %,Wind Absolute Appearance %,Wind Relative Appearance %
0,1940,2.731485e-09,1.000000,2.793287e-09,1.000000,1.084660e-09,1.000000
1,1941,2.756745e-09,1.009248,3.167608e-09,1.134007,1.089897e-09,1.004827
2,1942,2.743879e-09,1.004537,3.083440e-09,1.103875,1.157072e-09,1.066760
3,1943,2.774656e-09,1.015805,3.075601e-09,1.101069,1.440626e-09,1.328182
4,1944,2.788329e-09,1.020811,2.835732e-09,1.015196,1.480596e-09,1.365032
...,...,...,...,...,...,...,...
75,2015,1.111827e-08,4.070414,5.463863e-09,1.956069,1.383531e-09,1.275543
76,2016,1.074132e-08,3.932411,5.351956e-09,1.916006,1.426971e-09,1.315592
77,2017,1.051684e-08,3.850227,5.551843e-09,1.987566,1.437869e-09,1.325640
78,2018,1.025569e-08,3.754620,5.736526e-09,2.053683,1.327745e-09,1.224112


Google NGRAMS API request:

What are Google NGRAMS ?

NGRAM queries on Google NGRAM allows the user to see the percentage of appearance of such query amongst all the words accounted for in all the books present on Google Books.

We want to scrape data on rain, sun and wind in London.

Step 1
We started off by making a list of the queries we wanted to run for each rain, sun and wind. We alternated between capital and small letters when possible to collect as much perception data as possible.
LISTS OF QUERIES

We then used a for loop and the runQuery function to extract the appearance percentages from 1940 to 2019 of each query. The extracted data, being first in a json format, is then organised in a dataframe.
FUNCTION

We end up with 3 dataframes: NGRAMSrain_df, NGRAMSsun_df and NGRAMSwind_df.
EXAMPLE DATAFRAME RAIN

Step 2
In each dataframe, we summed up all different queries' perceptions for each year. This enabled us to have a general perception for a given year. The associated new dataframes wwere named NGRAMSrain_df_grouped, NGRAMSsun_df_grouped and NGRAMSwind_df_grouped.

Step 3
We created a new column which represents the relative appearance percentages. This was done by dividing percentages in each row by the first row. The column brings a clearer image of the change in these perceptions.
DATAFRAME RAIN WITH REL

Step 4
We merged the 3 dataframes into one general perception dataframe NGRAMS_df_grouped. It organises data on absolute and relative appearance percentages for rain, wind, and sun queries from 1940 to 2019.
FINAL DATAFRAME

# Create a SQL database using the following terminal commands:
- ```rm data/rainy.db``` Deletes the database if it is there (this is only run if we have updated our data)
- ```sqlite3 data/rainy.db``` Enters the SQL shell.
- ```.mode csv``` Changes mode to CSV.
- ```.import data/weather_data.csv weather``` Import each CSV as a new table.
- ```.import data/perception_data.csv perception```
- ```SELECT * FROM perception``` Check the data has loaded properly.
- ```SELECT * FROM weather```
- ```.quit``` Exits the SQL shell.

# Adjust the datatypes to the most efficient using the following SQL code run in the SQLTools VSCode Extension:
```SQL
CREATE TABLE new_weather (
    date DATE,
    city VARCHAR(18),
    temperature_2m_max DECIMAL(7,4),
    temperature_2m_min DECIMAL(7,4),
    temperature_2m_mean DECIMAL(7,4),
    daylight_duration DECIMAL(9,3),
    sunshine_duration DECIMAL(9,3),
    precipitation_sum DECIMAL(8,5),
    rain_sum DECIMAL(8,5),
    precipitation_hours TINYINT UNSIGNED
);

INSERT INTO new_weather (date, city, temperature_2m_max, temperature_2m_min, temperature_2m_mean,
                         daylight_duration, sunshine_duration, precipitation_sum, rain_sum, precipitation_hours)
SELECT date, city, temperature_2m_max, temperature_2m_min, temperature_2m_mean,
       daylight_duration, sunshine_duration, precipitation_sum, rain_sum, precipitation_hours
FROM weather;

DROP TABLE weather;

ALTER TABLE new_weather RENAME TO weather;

CREATE TABLE new_perception (
    Year YEAR,
    Perception DECIMAL(18,16)
);

INSERT INTO new_perception (Year, Perception) SELECT Year, Perception FROM perception;

DROP TABLE perception;

ALTER TABLE new_perception RENAME TO perception;
```

In [2]:
engine = create_engine('sqlite:///../data/rainy.db', echo=False, isolation_level="AUTOCOMMIT")

with engine.connect() as conn:
    pass

In [3]:
df = pd.read_sql('SELECT * FROM perception', engine)